<a href="https://colab.research.google.com/github/agungfirdaus717-ux/torentotgd/blob/main/SubTranslator2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================================
# 🔥 Google Colab - Multi SRT Translator
#    pakai deep-translator (Google Translate)
#    Tanpa API Key, auto-retry
# ========================================

!pip install deep-translator tqdm

import os, re, time, random
from deep_translator import GoogleTranslator
from tqdm import tqdm
from google.colab import files

# Translator siap pakai
translator = GoogleTranslator(source="auto", target="id")

def safe_translate(text_block, retries=9999, delay=1):
    """
    Translate dengan retry tanpa batas sampai sukses
    """
    for attempt in range(1, retries+1):
        try:
            result = translator.translate(text_block)
            if result:  # pastikan hasil tidak kosong
                return result
        except Exception as e:
            print(f"⚠️ Error attempt {attempt}: {e}")
            time.sleep(delay + random.uniform(0.2, 0.8))
    return text_block  # fallback terakhir (harusnya jarang dipakai)

def translate_srt(file_path, batch_size=20):
    """
    Translate file SRT ke bahasa Indonesia
    - batch_size: jumlah baris dialog per translate
    """
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    dialog_idx, dialog_lines = [], []
    for i, line in enumerate(lines):
        if re.match(r"^\d+$", line.strip()): continue
        if "-->" in line: continue
        if line.strip() == "": continue
        dialog_idx.append(i)
        dialog_lines.append(line.strip())

    translated = []
    for i in tqdm(range(0, len(dialog_lines), batch_size), desc=f"Translating {os.path.basename(file_path)}"):
        batch = dialog_lines[i:i+batch_size]
        text_block = "\n".join(batch)
        translated_text = safe_translate(text_block)
        translated.extend(translated_text.split("\n"))
        time.sleep(0.5)  # jeda kecil biar aman

    for idx, new_text in zip(dialog_idx, translated):
        lines[idx] = new_text + "\n"

    out_path = os.path.splitext(file_path)[0] + ".id.srt"
    with open(out_path, "w", encoding="utf-8") as f:
        f.writelines(lines)

    return out_path

# ========================================
# 🔼 Upload banyak file SRT
# ========================================
print("📂 Silakan upload file .srt (bisa banyak sekaligus)...")
uploaded = files.upload()

for fn in uploaded.keys():
    print(f"\n🔄 Proses: {fn}")
    out_file = translate_srt(fn, batch_size=20)
    print(f"✅ Selesai → {out_file}")
    files.download(out_file)
